In [3]:
import requests
import urllib
import time
import copy
import re
from bs4 import BeautifulSoup
import json

In [4]:
#1 Part
#1.1 Section
#1.1.1 Subsection
#1.1.1.1. Article
#	(1) Sentence
#		(a) Clause
#			(i) Subclause


#Actual dictionary structure looks like this:
#ArtX
    #ArticleX
    #SentenceX
        #ClauseX
            #SubclauseX

url = 'https://www.ontario.ca/laws/regulation/120332'
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, "html.parser")

<Response [200]>


In [5]:
#Make list of all the results for 'p' tags
#notice limit is just so we have a smaller list to test with
pTags = soup.find_all('p', class_ = ['section-e', 'subsection-e', 'clause-e', 'subclause-e'], limit = 30)

In [6]:
#create dictionary to contain dictionaries
p: dict = {}

#index for articles (for now), needs different regex expression than sentences, etc.
article = -1

In [7]:
def formatText(text): 
    
        text = str(text)
    
        cleaned = re.sub(r"<b>|</b>", "", text)
        cleaned = re.sub(r"<i>|</i>", "", cleaned)
        
        if (">" in cleaned):
              
            start = cleaned.index(">") + 1
            cleaned = cleaned[start:]
            
        if ("<" in cleaned):
        
            end = cleaned.index("<")
            cleaned = cleaned[:end]
        

        return cleaned

In [8]:
def label(list):
    
    labeled = []
    
    
    for item in list:
        
        if item.attrs['class'] == ['section-e']:
            
            cleaned = formatText(item)
            labeled.append({"type": "Section", "text": cleaned})
            
            
        elif item.attrs['class'] == ['subsection-e']:
            cleaned = formatText(item)
            labeled.append({"type": "Sub Section", "text": cleaned})
            
        elif item.attrs['class'] == ['clause-e']:
            cleaned = formatText(item)
            labeled.append({"type": "Clause", "text": cleaned})
            
        elif item.attrs['class'] == ['subclause-e']:
            cleaned = formatText(item)
            labeled.append({"type": "Sub Clause", "text": cleaned})
            
            
    return labeled

In [9]:
labeled = label(pTags)

In [10]:
def partitionPoints(list, type):
    
    points = []
    titles = []
    
    for item in list:
        
        if (item["type"] == type):
            points.append(list.index(item))
            titles.append(item)
        
    return [ points, titles ]

In [11]:
def partition(list, type):
    
    
    points = partitionPoints(list, type)[0]
    partitions = []
    
    for point in points:
        
        last_index = len(points) - 1
        current_index = points.index(point)
        
        if (last_index != current_index):
        
            start = point + 1
            end = points[current_index + 1]
            partition = list[start:end]
            
    
            
        else:
            
            start = point + 1
            partition = list[start:]
            
            
        partitions.append(partition)
    
    
    return partitions

In [12]:
def correlate(partitions, titles):
    
    list = []

    
    length = len(partitions)
    
    for i in range(length):
              
            
        title = titles[i]
        partition = partitions[i]
    
        
        list.append({"Section" : title, "Items" : partition})
              
    return list

In [13]:
p = partitionPoints(labeled, "Section")

In [14]:
section_partitions = partition(labeled, "Section")

In [33]:
section_partitions[0]

[{'type': 'Sub Section',
  'text': '  (1) Division A contains compliance and application provisions and the objectives and functional statements\r\nof this Code.'}]

In [16]:
r = correlate(section_partitions, p[1])

In [59]:
j = partitionPoints(labeled, "Sub Section")

In [62]:
sub_section_partitions = partition(labeled, "Sub Section")

In [63]:
sub_section_partitions

[[{'type': 'Section', 'text': '1.1.1.2. Scope of Division B'}],
 [{'type': 'Section', 'text': '1.1.1.3. Scope of Division C'}],
 [{'type': 'Section', 'text': '1.1.1.4. Internal Cross-references'}],
 [{'type': 'Section', 'text': '1.1.2.1. Application of Parts 1, 7 and 12'}],
 [],
 [{'type': 'Section', 'text': '1.1.2.2. Application of Parts 3, 4, 5 and 6'}],
 [{'type': 'Clause',
   'text': '  (a) used for major occupancies classified as,'},
  {'type': 'Sub Clause', 'text': '  (i) Group A, assembly occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group B, care, care and treatment or detention occupancies, or'},
  {'type': 'Sub Clause',
   'text': '  (iii) Group F, Division 1, high hazard industrial occupancies,'},
  {'type': 'Clause', 'text': '  (b) exceeding 600 m'},
  {'type': 'Sub Clause', 'text': '  (i) Group C, residential occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group D, business and personal services occupancies,'},
  {'type': 'Sub Clause', 'text': '  

In [105]:
def filterByType(l, type):

    arr = list(filter(lambda i: i["type"] != type, l))
    return arr

In [108]:
def cleanByType(l, t):
    
    clean = []
    
    for i in l:
        
        cleaned = filterByType(i, t)
        clean.append(cleaned)
        
    return clean

In [110]:
sub_section_partitions = cleanByType(sub_section_partitions, "Section")

In [111]:
sub_section_partitions

[[],
 [],
 [],
 [],
 [],
 [],
 [{'type': 'Clause',
   'text': '  (a) used for major occupancies classified as,'},
  {'type': 'Sub Clause', 'text': '  (i) Group A, assembly occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group B, care, care and treatment or detention occupancies, or'},
  {'type': 'Sub Clause',
   'text': '  (iii) Group F, Division 1, high hazard industrial occupancies,'},
  {'type': 'Clause', 'text': '  (b) exceeding 600 m'},
  {'type': 'Sub Clause', 'text': '  (i) Group C, residential occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group D, business and personal services occupancies,'},
  {'type': 'Sub Clause', 'text': '  (iii) Group E, mercantile occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (iv) Group F, Divisions 2 and 3, medium hazard industrial occupancies and low hazard industrial occupancies, or'},
  {'type': 'Clause', 'text': '  (c) used for retirement homes.'}],
 [{'type': 'Clause', 'text': '  (a)  post-disaster buildings,'},
 

In [206]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(r, f, ensure_ascii=False, indent=4)

In [119]:
def cleanText(list): 
    
    cleaned = []
    
    for i in list:
        clean_text = formatText(i)
        cleaned.append(clean_text)
        
    return cleaned

In [52]:



arr = []

for i in r:
    
    items = i["Items"]
    print(items)
    group = partition(items,"Sub Section" )
    arr.append(group)

[{'type': 'Sub Section', 'text': '  (1) Division A contains compliance and application provisions and the objectives and functional statements\r\nof this Code.'}]
[{'type': 'Sub Section', 'text': '  (1) Division B contains the acceptable solutions\r\nof this Code.'}]
[{'type': 'Sub Section', 'text': '  (1) Division C contains the administrative provisions of this Code.'}]
[{'type': 'Sub Section', 'text': '  (1) If a provision of this Code contains a reference to another provision of this Code but no Division is specified, both provisions are in the same Division of this Code.'}]
[{'type': 'Sub Section', 'text': ' (1) Part 1 of Division B applies to all buildings.'}, {'type': 'Sub Section', 'text': ' (2) Subject to Article 1.1.2.6., Parts 7 and 12 of Division B apply to all buildings.'}]
[{'type': 'Sub Section', 'text': ' (1) Subject to Articles 1.1.2.6. and 1.3.1.2., Parts 3, 5 and 6 of Division B apply to all buildings,'}, {'type': 'Clause', 'text': '  (a) used for major occupancies c

In [56]:
arr[5]

[[{'type': 'Clause',
   'text': '  (a) used for major occupancies classified as,'},
  {'type': 'Sub Clause', 'text': '  (i) Group A, assembly occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group B, care, care and treatment or detention occupancies, or'},
  {'type': 'Sub Clause',
   'text': '  (iii) Group F, Division 1, high hazard industrial occupancies,'},
  {'type': 'Clause', 'text': '  (b) exceeding 600 m'},
  {'type': 'Sub Clause', 'text': '  (i) Group C, residential occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (ii) Group D, business and personal services occupancies,'},
  {'type': 'Sub Clause', 'text': '  (iii) Group E, mercantile occupancies,'},
  {'type': 'Sub Clause',
   'text': '  (iv) Group F, Divisions 2 and 3, medium hazard industrial occupancies and low hazard industrial occupancies, or'},
  {'type': 'Clause', 'text': '  (c) used for retirement homes.'}],
 [{'type': 'Clause', 'text': '  (a)  post-disaster buildings,'},
  {'type': 'Clause', 'text': ' 

In [49]:
def part(list, type):


    for i in list:

        items = i["Items"]
        print(items)

        group = partition(items,"Sub Section")
        titles = partitionPoints(items,"Sub Section")[1]
        
        print(titles)


In [51]:
part(arr, "Sub Section")

TypeError: list indices must be integers or slices, not str